In [1]:
def OCRImgLines(imagePath,folder):
    print(imagePath)
    image = cv2.imread(imagePath)
    text = str(pytesseract.image_to_string(image))
    #print(text)

    text_file = open(folder+'\\'+file[:-4]+'.txt', "w")
    n = text_file.write(text)
    text_file.close()
    text_file = open(folder+'\\'+file[:-4]+'.txt', 'r')
    lines = text_file.readlines()
    text_file.close()
    return lines

def checkVar(toCheck):
    if toCheck is None:
        return ''
    else:
        return toCheck

In [2]:
def extractNLine(keyphrase,lines,nextPageImagePath,folder,n=1):

    catchPhrases=['Comment\n',
    'Scope 2, market-based\n'
    ]

    data = None
    for i in range(len(lines)):
        #if keyphrase is in ;ine, the next line contains the data
        #print('len(lines): '+str(len(lines)))
        #print('i+1: '+str(i))
        try:
            if keyphrase in lines[i] and (len(lines)>=i+n):
                data=lines[i+n]
            #if keyphrase is in last line, the first line in the next page contains the data
            elif keyphrase in lines[i] and (len(lines)==i+n):
                print('Keyphrase found in last line: taking first line of next page')
                nextPageLines = [line for line in OCRImgLines(nextPageImagePath,folder) if line!='\n']
                for j in range(len(nextPageLines)):
                    if 'DISCLOSURE INSIGHT ACTION' in nextPageLines[j]:
                        data=nextPageLines[j+1]
        except IndexError:
                    data=''  
            
    print(data)         
    if data == 'Scope 2, market-based\n':
        print('FOUND ERROR')
    '''
    If the keyphrase is not in the first page, it checks the second page
    does this until it finds data. 
    '''
    if data == None or (data in catchPhrases):
        print('data not found in page, checking next page')
        print(keyphrase)
        nextPageLines = [line for line in OCRImgLines(nextPageImagePath,folder) if line!='\n']
        #print(nextPageLines)
        for i in range(len(nextPageLines)):
            #print(i)
            print(nextPageLines[i])
            if keyphrase in nextPageLines[i] :
                data = nextPageLines[i+n]
    return data

'''def extractBaseYear(keyphrase,lines,nextPageImagePath,folder):
    print('Extracting Base Year')
    for i in range(len(lines)):
        try:
            if 'Scope 1' in lines[i] and (len(lines)>=i+1):
        except IndexError:
                data='''''


def dataExtract(toExtract,folder,file):
    imagePath = (folder+'\\'+file)
    nextPageImagePath = (folder+'\\1-'+file[2:])

    #extracts the lines of text from an image
    lines = [line for line in OCRImgLines(imagePath,folder) if line!='\n']
    lines=lines[:-1]#removes pagenumber
    print(lines)
    data = None

    #sets the keyphrase depending on the data found in the page
    if toExtract == 'scope1':
        keyphrase = "Gross global Scope 1 emissions (metric tons CO2e)"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder)

    elif toExtract == 'scope2L':
        keyphrase = "Scope 2, location-based"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder)
    
    elif toExtract == 'scope2M':
        keyphrase = "Scope 2, market-based"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder)
    

    elif toExtract == 'scope3':
        keyphrase = "Metric tonnes CO2e"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder)
        

    elif toExtract == "baseYear":
        keyphrase = "Base year start"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder)

    elif toExtract == "byScope1":
        keyphrase = "Scope 1"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder,n=6)

    elif toExtract == "byScope2L":
        keyphrase = "Scope 2 (location-based)"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder,n=6)

    elif toExtract == "byScope2M":
        keyphrase = "Scope 2 (market-based)"
        data = extractNLine(keyphrase,lines,nextPageImagePath,folder,n=6)


    '''
    Goes through each line in the page
    if the keyphrase is in the line, the next line will contain the data we want
    saves the next line as the data

    TODO Fix bug with label on last line, and value on first line of next page ^Microsoft
    '''
    
    
    
                    

    return data

In [3]:
from pathlib import Path
def OCRPages(folderpath,pages,numPages):
    Path(folderpath+'\\bin').mkdir(parents=True, exist_ok=True)
    for i in reversed(range(0, numPages)):
        print('---checking page '+str(i))
        pages[i].save(folderpath+'\\bin\\'+str(i)+'.png')
        lines = [line for line in OCRImgLines((folderpath+'\\bin\\'+str(i)+'.png'),folderpath+'\\bin\\') if line!='\n']
        for phrase in filterPhrases:
            for line in lines:
                if phrase in line:
                    pages[i].save(folderpath+'\\0-'+phrase+'.png')
                    pages[i+1].save(folderpath+'\\1-'+phrase+'.png')
    for f in os.listdir(folderpath+'\\bin'):
        os.remove(folderpath+'\\bin\\'+f)
    os.rmdir(folderpath+'\\bin')

In [4]:
import cv2
import os
import re
import pytesseract
from pdf2image import convert_from_path
from os import walk
import csv
import PyPDF2
import warnings
warnings.filterwarnings("error")


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
filteredPages = []
filterPhrases = ['C6.1',
                'C6.3',
                'C6.5',
                'C5.1']
count=0
dates = ['2019','2020','2021','2022']

folder = r'C:\Users\Clamfighter\Documents\GitHub\Data-Mining\Data-Mining\CDP\pdfs(unf)'

all_files = os.listdir(folder)    
pdf_files = list(filter(lambda f: f.endswith('.pdf'), all_files))

for file in pdf_files:
    companyName=''
    date=''
    try:
        print(file)
        pdfname = file
        folderpath = folder+"\\"+file[:-4]
        print(folderpath)
        if not os.path.exists(folderpath):
            os.makedirs(folderpath)

        #convert the pages to images
        print('converting Pages')
        pages = convert_from_path(folder+"\\"+file, 350)
        print('pages Converted')
        

        '''
        Searching of PDF for wanted data
        '''
        print('Checking Pages for wanted data')
        pdf = PyPDF2.PdfFileReader(folder+"\\"+file,strict=False)

        '''
        Checking first page of data
        '''
        #print(pages)
        pages[0].save(folderpath+'\Page0.png')
        lines = [line for line in OCRImgLines((folderpath+'\Page0.png'),folderpath) if line!='\n']
        #print(lines)
        openingLines = lines[0:3]
        #print(openingLines)
        if any('Climate Change' in line for line in openingLines):
            print('Correct PDF Format')
            sep = 'Climate Change'
            for openingLine in reversed(openingLines):
                for d in dates:
                    if d in openingLine:
                        date=d
                print(openingLine)
                if sep in openingLine:
                    print('Opening Line Contains Name')
                    companyName = openingLine.split(sep, 1)[0]
                    print(companyName)
                    companyName = re.sub('CDP', '', companyName)
                    print(companyName)
        else:
            print('Incorrect format found due to lack of "Climate Change" in first lines of first page')
            continue
        print(companyName)
        print(date)
        
        foundDate=False #used to indicate whether any data was found in any of the pages
        textExtracted=False
        try:
            numPages = pdf.getNumPages()
            for i in reversed(range(0, numPages)):
                print('---checking page '+str(i))
                pageObj = pdf.getPage(i)
                text = pageObj.extractText()
                #print(text)
                for phrase in filterPhrases:
                    #print('---searching for phrase '+phrase)
                    #resSearch = re.search(phrase, text)
                    
                    if phrase in text:
                        #print('Data Found: ', phrase)
                        
                        pages[i].save(folderpath+'\\0-'+phrase+'.png')
                        pages[i+1].save(folderpath+'\\1-'+phrase+'.png')
                        foundData=True
                
                if text!='':
                    textExtracted=True
                        
        except NotImplementedError                      :
            print('PDF Encrypted')
        except Exception as e:
            print(e)
            numPages=len(pages)
            OCRPages(folderpath,pages,numPages)
            textExtracted=True


        if not textExtracted:
            print('Uable to extract text from PDF file. Conducting OCR. This may take some minutes')
            OCRPages(folderpath,pages,numPages)

        print('Data pages Extracted')
        print('Extracting data from pages')
        scope1=''
        scope2L=''
        scope2M=''
        scope3=''
        baseYear=''
        byScope1=''
        byScope2L=''
        byScope2M=''


        for file in os.listdir(folderpath):
            if file == '0-C6.1.png':
                scope1 = checkVar(dataExtract('scope1',folderpath,file))
            elif file == '0-C6.3.png':
                scope2L = checkVar(dataExtract('scope2L',folderpath,file))
                scope2M = checkVar(dataExtract('scope2M',folderpath,file))

            elif file == '0-C6.5.png':
                scope3 = checkVar(dataExtract('scope3',folderpath,file))
            elif file == '0-C5.1.png':
                baseYear = checkVar(dataExtract('baseYear',folderpath,file))
                byScope1 = checkVar(dataExtract('byScope1',folderpath,file))
                byScope2L = checkVar(dataExtract('byScope2L',folderpath,file))
                byScope2M = checkVar(dataExtract('byScope2M',folderpath,file))
                
        print(scope1)
        print(scope2L)
        print(scope2M)
        print(scope3)
        print(baseYear)
        print(byScope1)
        print(byScope2L)
        print(byScope2M)
        print('Exporting Data')
        row=[date,scope1,scope2L,scope2M,scope3,baseYear,byScope1,byScope2L,byScope2M]
        row = [item.strip() for item in row]
        row = [item.replace(' ', '') for item in row]
        row.append(companyName)
        row.append(pdfname)
        f = open(folder+'/output.csv', 'a', newline='')
        writer = csv.writer(f)
        writer.writerow(row)
        f.close()
    except (FileNotFoundError, NotImplementedError,IndexError) as e:
        print('error: '+str(e))
        row=[date,'','','',companyName,pdfname]
        f = open(folder+'/output.csv', 'a', newline='')
        writer = csv.writer(f)
        writer.writerow(row)
        f.close()

2019+CDP+Report.pdf
C:\Users\Clamfighter\Documents\GitHub\Data-Mining\Data-Mining\CDP\pdfs(unf)\2019+CDP+Report
converting Pages
pages Converted
Checking Pages for wanted data
C:\Users\Clamfighter\Documents\GitHub\Data-Mining\Data-Mining\CDP\pdfs(unf)\2019+CDP+Report\Page0.png
Correct PDF Format
CO. Introduction

DISCLOSURE INSIGHT ACTION

ManpowerGroup - Climate Change 2019 q : CDP

Opening Line Contains Name
ManpowerGroup - 
ManpowerGroup - 
ManpowerGroup - 
2019
---checking page 43
---checking page 42
---checking page 41
---checking page 40
---checking page 39
---checking page 38
---checking page 37
---checking page 36
---checking page 35
---checking page 34
---checking page 33
---checking page 32
---checking page 31
---checking page 30
---checking page 29
---checking page 28
---checking page 27
---checking page 26
---checking page 25
---checking page 24
---checking page 23
---checking page 22
---checking page 21
---checking page 20
---checking page 19
---checking page 18
---checkin